use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import random
import numpy as np
import pandas as pd
# import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle


/root/autodl-tmp/xiaolong/anaconda3/envs/pytorch_gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [3]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [4]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 2048
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), dtype=torch.long, device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id].to('cuda:2')
            product_emb = product_embeddings[batch_product_id].to('cuda:2')
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [5]:
def normalize_scores(score_df, score_name, normalized_score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df['exp_score'] = np.exp(score_df[score_name].to_numpy())
    scores_sum = score_df[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum.reset_index(inplace=True)
    scores_sum = scores_sum.sort_values(by=['sess_id'], ascending=True)
    scores_sum.reset_index(drop=True, inplace=True)
    scores_sum.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df = score_df.merge(scores_sum, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df = merged_score_df.sort_values(by=['sess_id', 'product'])
    merged_score_df.reset_index(drop=True, inplace=True)
    
    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    # del scores_sum_g
    # del merged_score_df_g  

# Merge test score

In [6]:
FIELD_NAME = 'gru4rec_scores_2'

In [7]:
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_test_feature.parquet'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1_phase2.csv'

In [8]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_DE/product_embeddings_2023-06-01-20-52-24.pt'
DE_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_DE/predict_embeddings_2023-06-01-20-53-05.pt'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_JP/product_embeddings_2023-06-01-20-53-24.pt'
JP_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_JP/predict_embeddings_2023-06-01-20-54-09.pt'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_UK/product_embeddings_2023-06-01-20-54-25.pt'
UK_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/GRU4Rec_Next/kdd_cup_2023_UK/predict_embeddings_2023-06-01-20-55-08.pt'

In [9]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_test_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [10]:
merged_candidates = read_merged_candidates_feature_test()
test_sessions = read_test_sessions()
EMBED_DIM = 128
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [38]:
# sess embeddings 
test_DE_query_emb = torch.load(DE_test_embeddings_path, map_location='cpu')
test_JP_query_emb = torch.load(JP_test_embeddings_path, map_location='cpu')
test_UK_query_emb = torch.load(UK_test_embeddings_path, map_location='cpu')
test_query_embeddings = torch.cat(
    [test_DE_query_emb[test_sessions['locale'] == 'DE'], test_JP_query_emb[test_sessions['locale'] == 'JP'], test_UK_query_emb[test_sessions['locale'] == 'UK']],
    dim=0)

In [39]:
# product embeddings 
DE_product_emb = torch.load(DE_product_embeddings_path, map_location='cpu')
JP_product_emb = torch.load(JP_product_embeddings_path, map_location='cpu')
UK_product_emb = torch.load(UK_product_embeddings_path, map_location='cpu')
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [40]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/b5aeac4e5b9ff0518bbcb59a28086594'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/2536617955df215e0047f5b220d1c012'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/8d133ea55ad67bd3efd625dfeff0fb1d'
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [41]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [42]:
# merged_candidates_g = cudf.from_pandas(merged_candidates)
# product_id_df_g = cudf.from_pandas(product_id_df)

In [43]:
# merged_candidates_gru4rec_g = merged_candidates_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
# merged_candidates_gru4rec_g['dataset_id'] = merged_candidates_gru4rec_g['dataset_id'].fillna(0)
# merged_candidates_gru4rec_g.drop(columns=['locale'], inplace=True)
# merged_candidates_gru4rec_g = merged_candidates_gru4rec_g.sort_values(by=['sess_id', 'product'])
# merged_candidates_gru4rec_g.reset_index(drop=True, inplace=True)
# merged_candidates_gru4rec = merged_candidates_gru4rec_g.to_pandas()

In [ ]:
merged_candidates_gru4rec = merged_candidates.merge(product_id_df, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_gru4rec['dataset_id'] = merged_candidates_gru4rec['dataset_id'].fillna(0)
merged_candidates_gru4rec.drop(columns=['locale'], inplace=True)
merged_candidates_gru4rec = merged_candidates_gru4rec.sort_values(by=['sess_id', 'product'])
merged_candidates_gru4rec.reset_index(drop=True, inplace=True)
# merged_candidates_gru4rec = merged_candidates_gru4rec_g.to_pandas()

In [44]:
# del merged_candidates_g
# del product_id_df_g
# del merged_candidates_gru4rec_g

In [45]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_gru4rec['dataset_id'][merged_candidates_gru4rec['sess_locale'] == locale] = \
        merged_candidates_gru4rec['dataset_id'][merged_candidates_gru4rec['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_712375/4111213791.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_gru4rec['dataset_id'][merged_candidates_gru4rec['sess_locale'] == locale] = \


In [46]:
merged_candidates_gru4rec[FIELD_NAME] = get_scores(merged_candidates_gru4rec, test_query_embeddings, product_embeddings)

100%|██████████| 33901/33901 [00:34<00:00, 969.07it/s] 


In [47]:
normalize_scores(merged_candidates_gru4rec, FIELD_NAME, 'normalized_'+FIELD_NAME)

In [48]:
merged_candidates_gru4rec.drop(columns=['dataset_id', 'exp_score', 'score_sum'], inplace=True)

In [52]:
cast_dtype(merged_candidates_gru4rec)
merged_candidates_gru4rec.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [60]:
merged_candidates

,sess_id,sess_locale,product,sasrec_scores_2,sasrec_normalized_scores_2
0,0,DE,4088833651,0.000000,2.975813e-09
1,0,DE,B000H6W2GW,0.000000,2.975813e-09
2,0,DE,B000JG2RAG,7.665308,6.347557e-06
3,0,DE,B000RYSOUW,-2.951060,1.555882e-10
4,0,DE,B000UGZVQM,3.977920,1.589257e-07
...,...,...,...,...,...
69428426,316970,UK,B0BJCTH4NH,11.327528,1.041200e-04
69428427,316970,UK,B0BJTQQWLG,5.604142,3.403292e-07
69428428,316970,UK,B0BJV3RL4H,9.146974,1.176336e-05
69428429,316970,UK,B0BK7SPC84,-10.383047,3.879279e-14


In [51]:
merged_candidates_gru4rec

,sess_id,sess_locale,product,sasrec_scores_2,sasrec_normalized_scores_2,gru4rec_scores,gru4rec_normalized_scores,product_freq,sess_avg_price,product_price,gru4rec_scores_2,gru4rec_normalized_scores_2
0,0,DE,4088833651,0.000000,2.975813e-09,0.000000,1.580065e-09,828,25.195269,36.761604,0.000000,0.0
1,0,DE,B000H6W2GW,0.000000,2.975813e-09,0.000000,1.580065e-09,875,25.195269,36.761604,0.000000,0.0
2,0,DE,B000JG2RAG,7.665308,6.347557e-06,8.104032,5.226502e-06,24,25.195269,23.190001,11.372551,0.000115
3,0,DE,B000RYSOUW,-2.951060,1.555882e-10,-2.857798,9.068785e-11,5,25.195269,6.900000,-2.205641,0.0
4,0,DE,B000UGZVQM,3.977920,1.589257e-07,4.688567,1.717488e-07,4,25.195269,21.990000,8.559400,0.000007
...,...,...,...,...,...,...,...,...,...,...,...,...
69428426,316970,UK,B0BJCTH4NH,11.327528,1.041200e-04,10.629994,3.818184e-04,74,16.950001,5.800000,11.301320,0.000264
69428427,316970,UK,B0BJTQQWLG,5.604142,3.403292e-07,6.052083,3.923694e-06,6,16.950001,9.880000,8.246040,0.000012
69428428,316970,UK,B0BJV3RL4H,9.146974,1.176336e-05,7.667603,1.973815e-05,7,16.950001,22.097065,9.860847,0.000062
69428429,316970,UK,B0BK7SPC84,-10.383047,3.879279e-14,-6.356799,1.601719e-11,0,16.950001,5.960000,-7.227418,0.0


In [54]:
merged_candidates_gru4rec[(merged_candidates_gru4rec['sess_id'] == 100005)].sort_values(by='normalized_sasrec_scores_2', ascending=False)[
    ['sess_locale', 'product', 'normalized_sasrec_scores_2', 'sasrec_scores_2', 'normalized_gru4rec_scores_2', 'gru4rec_scores_2']
][:15]

,sess_locale,product,sasrec_normalized_scores_2,sasrec_scores_2,gru4rec_normalized_scores,gru4rec_scores,gru4rec_normalized_scores_2,gru4rec_scores_2
21586793,DE,B07TRQH45S,0.157271,14.288859,0.252197,13.236230,0.129736,12.857471
21586762,DE,B01N4ND1T2,0.122860,14.041931,0.073287,12.000405,0.121831,12.794599
21586937,DE,B0B6WNV91T,0.109754,13.929132,0.063878,11.863000,0.124358,12.815129
21586840,DE,B092CMLDHW,0.084313,13.665420,0.036893,11.314045,0.034603,11.535893
21586866,DE,B09F2J37V4,0.066638,13.430163,0.021890,10.792058,0.085040,12.435093
21586887,DE,B09NQ7T1D2,0.044435,13.024924,0.016852,10.530479,0.015521,10.734191
21586870,DE,B09F66MWVX,0.035420,12.798150,0.015094,10.420308,0.018534,10.911561
21586934,DE,B0B62K5H9P,0.023618,12.392890,0.021667,10.781809,0.030203,11.399909
21586846,DE,B0953JXVQ2,0.021310,12.290046,0.012762,10.252530,0.007571,10.016322
21586824,DE,B08H1YNK3P,0.018914,12.170765,0.028193,11.045082,0.015597,10.739016
